# LAT Workshop Tutorial #2: EMRIs and LISA Response + TDI

张铭昊 北京化工大学 zmh780674484@163.com

In [ ]:
LISA数据分析工具研讨会（LAT）教程2：极端质量比例旋与LISA的响应+TDI技术

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from lisatools.utils.constants import *
from copy import deepcopy  # can be useful

In the second tutorial, we will

* generate EMRI waveforms,
* use tools from our last lesson to estimate EMRI SNRs,
* examine changing EMRI trajectories,
* add in the LISA response function,
* and calculate Likelihoods for EMRIs + Response + Trajectory. 

In [ ]:
在第二个教程中，我们将
产生极端质量比例旋系统的引力波波形
使用上一节课学到的方法来估计测量极端质量比例旋系统时的信噪比
研究和分析随时间变化的极端质量比例旋系统（EMRI）轨迹
添加LISA响应函数
计算极端质量比例旋系统的似然值、响应度、轨迹

## Task 1: Generate an EMRI Waveform for a typical EMRI

In [ ]:
任务1：生成典型极端质量比例旋系统（EMRI）的引力波波形

Generate a relativistic Schwarzschild eccentric EMRI Waveform in the detector frame (use `GenerateEMRIWaveform` class rather than `FastSchwarzschildEccentricFlux`. Choose a set of reasonable parameters.

Useful documentation:
* [GenerateEMRIWaveform](https://bhptoolkit.org/FastEMRIWaveforms/html/user/main.html#few.waveform.GenerateEMRIWaveform)
* [FastSchwarzschildEccentricFlux](https://bhptoolkit.org/FastEMRIWaveforms/html/user/main.html#few.waveform.FastSchwarzschildEccentricFlux)

In [ ]:
在探测器参考框架中生成一个相对论性的史瓦西偏心EMRI引力波信号的波形。（使用GenerateEMRIWaveform
类而不是FastSchwarzschildEccentricFlux.选择一套合理的参数

In [ ]:
# imports
from few.waveform import GenerateEMRIWaveform

In [ ]:
gen_wave = GenerateEMRIWaveform("FastSchwarzschildEccentricFlux")


T = 0.1  
dt = 10.0  
M = 1e6
Phi_phi0 = 1.0
Phi_theta0 = 2.0
Phi_r0 = 3.0
a = 0.1  
mu = 1e1
p0 = 12.0
e0 = 0.3
x0 = 1.0  
qK = 0.2  
phiK = 0.1  
qS = 0.25  
phiS = 0.2  
dist = 1.0 
h = gen_wave(
    M,
    mu,
    a,
    p0,
    e0,
    x0,
    dist,
    qS,
    phiS,
    qK,
    phiK,
    Phi_phi0,
    Phi_theta0,
    Phi_r0,
    T=T,
    dt=dt,
)

plt.plot(np.arange(2000) * dt, h.real[:2000])

### Question

Why does the EMRI waveform look like that? This is obviously a very open-ended question. Try to decipher why the waveform has such a structure in terms of the compact object orbits.

In [ ]:
问题
为什么EMRI的引力波波形是那个样子？这显然是一个非常开放性的问题。试着解释为什么在
微小轨道时引力波波形具有这样的形式

## Task 2: Add the LISA response function

When actually building the LISA response function in the time-domain, the edges of the data can be complicated. This is due to the various delays included in the instrument response and Time Delay Interferometry (TDI) transforms. In short, summations between the waveform at the current time and times in the past require that we start our computations a bit after the beginning of observation: if you cannot reach into the past enough (i.e. the delay causes the time to be less than 0.0).

Now we will wrap the EMRI waveforms in LISA response using `ResponseWrapper` from `fastlisasresponse`.

Useful documentation:
* [ResponseWrapper](https://mikekatz04.github.io/lisa-on-gpu/html/user/main.html#fastlisaresponse.response.ResponseWrapper)
* [pyResponseTDI](https://mikekatz04.github.io/lisa-on-gpu/html/user/main.html#fastlisaresponse.response.pyResponseTDI)
* [EqualArmlengthOrbits](https://mikekatz04.github.io/LISAanalysistools/build/html/user/main.html#lisatools.detector.EqualArmlengthOrbits)

In [ ]:
任务2：添加LISA响应函数
实际上在建立时域的LISA响应函数的过程中，数据的边界可能是复杂的。这是因为一些仪器响应和时间延迟干涉（TDI）变换中包含的各种延迟导致的。
简而言之，当前时间和过去时间点处的波形之间的求和需要我们在观测开始后一小段时间开始计算：如果无法回溯到足够远的过去（即延迟导致时间小于0.0）。
现在我们将使用fastlisasresponse中的ResponseWrapper对LISA响应中的EMRI波形进行打包

In [ ]:
# imports
from fastlisaresponse import ResponseWrapper
from lisatools.detector import EqualArmlengthOrbits

In [ ]:
sampling_frequency = 1 / dt
t0 = 20000.0

order = 25

orbits = EqualArmlengthOrbits()


tdi_gen = "prime"


index_beta = 6
index_lambda = 7
tdi_kwargs = dict(
    order=order, tdi=tdi_gen, tdi_chan="Ae",
)

emri_lisa = ResponseWrapper(
    gen_wave,
    T,
    dt,
    index_lambda,
    index_beta,
    t0=t0,
    orbits=orbits,
    
)

Ae = emri_lisa(
    M,
    mu,
    a,
    p0,
    e0,
    x0,
    dist,
    qS,
    phiS,
    qK,
    phiK,
    Phi_phi0,
    Phi_theta0,
    Phi_r0,
)
emri_lisa.response_model.num_total_points
start_ind = int(t0 / dt)
num = 500
fig, ax = plt.subplots(3, 1, sharex=True)

for i in range(3):
    ax[i].plot((start_ind + np.arange(num)) * dt, Ae[i][:num], color=f"C{i}")


### Question

With a long-enough observation time, you will see a new envelope structure to the waveforms when the response is added. Why is that? 

If you want to use `ResponseWrapper` with another type of waveform, the above cells are how you do that. If you want to use `GenerateEMRIWaveform` and `ResponseWrapper` together, you can use the stock model from LISA Analysis Tools: [EMRITDIWaveform](https://mikekatz04.github.io/LISAanalysistools/build/html/user/sources.html#lisatools.sources.emri.EMRITDIWaveform). With this class, you do not have to set any choices. In this case, it will set it up with all defaults. Generate the same waveform above but now with the new class. 

Documentation:
* [EMRITDIWaveform](https://mikekatz04.github.io/LISAanalysistools/build/html/user/sources.html#lisatools.sources.emri.EMRITDIWaveform)

In [ ]:
问题
在足够长的观测时间下，当响应函数添加后你将看到一个新的包络结构。为什么是那样？
若你想同时使用GenerateEMRIWaveform和GenerateEMRIWaveform，你可以使用LISA分析工具中含有的模型EMRITDIWaveform，在使用它们时，你不需要设定
任何选项，此时它会自动以所有默认选项设置好。现在请生成一个跟上面一样的波形，但是使用这个新工具。


In [ ]:
# imports
from lisatools.sources.emri import EMRITDIWaveform

In [ ]:
emri_lisa = EMRITDIWaveform(response_kwargs=dict(tdi_chan="AE"))
AE = emri_lisa(
    M,
    mu,
    a,
    p0,
    e0,
    x0,
    dist,
    qS,
    phiS,
    qK,
    phiK,
    Phi_phi0,
    Phi_theta0,
    Phi_r0,
)
fig, ax = plt.subplots(2, 1, sharex=True)

for i in range(2):
    ax[i].plot(AE[i])

## Task 3: Calculate SNR, inner product, and Likelihood

Now we will combine our knowledge from tutorials 1 and 2 to calculate the SNR, inner product, and Likelihood for an EMRI.

Let's start by generating our data for *just* the A and E channels (do not worry about T for now). Be careful, make sure when using `EMRITDIWaveform`, you initialize with the keyword argument `response_kwargs=dict(tdi_chan="AE")` to remove the T channel. Fill a [DataResidualArray](https://mikekatz04.github.io/LISAanalysistools/build/html/user/datacontainer.html#lisatools.datacontainer.DataResidualArray) with the data. Plot it if you would like. 

In [ ]:
任务3：计算信噪比、内积、似然值

In [ ]:
现在我们将结合教程1、2的知识来计算极端质量比例旋系统的信噪比、内积和似然值。
让我们通过仅在A,E通道（现在不用担心T通道）生成数据来开始。谨慎一点，要确保当使用EMRITDIWaveform时，你要使用response_kwargs=dict(tdi_chan="AE")
关键字进行初始化以清空T通道。用这些数据填入DataResidualArray，如果你想的话可以画出图来

In [ ]:
from lisatools.datacontainer import DataResidualArray

In [ ]:
AE = emri_lisa(
    M,
    mu,
    a,
    p0,
    e0,
    x0,
    dist,
    qS,
    phiS,
    qK,
    phiK,
    Phi_phi0,
    Phi_theta0,
    Phi_r0,
)

data = DataResidualArray(AE, dt=dt)

Setup the sensitivity matrix: [AE1SensitivityMatrix](https://mikekatz04.github.io/LISAanalysistools/build/html/user/sensitivity.html#lisatools.sensitivity.AE1SensitivityMatrix)

In [ ]:
设置灵敏度矩阵：AE1SensitivityMatrix

In [ ]:
from lisatools.sensitivity import AE1SensitivityMatrix

Now pack them both into the [AnalysisContainer](https://mikekatz04.github.io/LISAanalysistools/build/html/user/datacontainer.html#lisatools.analysiscontainer.AnalysisContainer) and calculate the SNR of the injection data.

In [ ]:
现在，把它们装入AnalysisContainer并计算输入数据的信噪比

In [ ]:
from lisatools.analysiscontainer import AnalysisContainer

In [ ]:
analysis = AnalysisContainer(data, sens_mat, signal_gen=emri_lisa)
analysis.loglog()
analysis.snr()

Now, let's calculate the Likelihood using `AnalysisContainer.calculate_signal_likelihood`. This allows us to compare another EMRI signal to our injection. Start by injecting a source with the correct injection parameters. What is the Likelihood here? Now change the source parameters slightly one at a time and observe how the Likelihood changes. This can give you a sense of the complicated multidimensional Likelihood surface which we wish to learn. Since we are fixing the sensitivity for now, add the `source_only=False` keyword argument. 

In [ ]:
  现在，让我们使用AnalysisContainer.calculate_signal_likelihood 语句计算似然值。这让我们能够用另一个极端质量比例旋系统的信号与我们所输
入的来进行比较。输入一个有着正确输入参数的源。这里的似然值是多少？现在一个个地略微改变源的参数并观察似然值怎样改变。这可以给我们带来对我们
想要了解的复杂多维度似然值的一种理解。由于我们固定了灵敏度，因此要添加 "source_only=False" 关键字参数。

In [ ]:
analysis.calculate_signal_likelihood(
    M * 1.00000005,
    mu,
    a,
    p0,
    e0,
    x0,
    dist,
    qS,
    phiS,
    qK,
    phiK,
    Phi_phi0,
    Phi_theta0,
    Phi_r0,
    source_only=True
)

## Task 4: Determine 1$\sigma$ width in 1-dimension using the Likelihood

In one dimension, the 1$\sigma$ Likelihood width can be determined by solving $\log{\mathcal{L(\vec{\theta})}} = -1$. Using a loop, plot the Likelihood versus one parameter you decide to change. Make sure the Likelihoods are all of order $\sim1$, so close to the true value. To make sure this works as an example, it is advisable to use just one mode by setting `waveform_kwargs=dict(mode=[(2, 2, 0)]` (you can use any mode). 

In [ ]:
任务4：确定1维空间使用的似然值对应的1σ区间  

In [ ]:
在一个维度中，1σ似然值区间被方程log(θ）=−1确定。使用循环语句，绘制对应于你进行改动的一个参数的似然值。确保似然值接近1，与真实值接近。
为了确保这个例子的准确性，我建议仅使用1个模式，设定waveform_kwargs=dict(mode=[(2, 2, 0)]（你可以使用任何模式）                     

In [ ]:
x = np.linspace(-0.01, 0.01, 20)
y = np.zeros_like(x)
for i in range(len(x)):
    diff = 1.0 + x[i]
    y[i] = analysis.calculate_signal_likelihood(
        M,
        mu,
        a,
        p0,
        e0,
        x0,
        dist * diff,
        qS,
        phiS,
        qK,
        phiK,
        Phi_phi0,
        Phi_theta0,
        Phi_r0,
        source_only=True,
        waveform_kwargs=dict(modes=[(2, 2, 0)])
    )


Use whatever method you prefer to determine the $1\sigma$ width. You can eyeball it or use plotting. We are just estimating the value for now. 

In [ ]:
使用一切你喜欢的方法确定1σ区间。你可以使用观察或绘图法。我们只需要在当前阶段估计这个值。

In [ ]:
mean = np.mean(data)
std = np.std(data)

sigma_width = 2 * std

print("Estimated 1sigma width:", sigma_width)

## Task 5: analyze a GR modification or environmental effect

In [ ]:
任务5：分析广义相对论效应或环境影响带来的修正

Now we will look at a very useful aspect of `few`: testing beyond-GR or environmental effects by adding terms to the Trajectory. Here, we will use a modified trajectory that we have already put together for you. The modificiation is `pdot = pdot * (1 + mod)` and `edot = edot * (1 + mod)`.

It is imported below (`ModifiedPnTrajectory`). It is a basic PN trajectory in the Schwarzschild eccentric limit. This is effectively lifted from the `few` tutorial (adding the GR modification). For the amplitude module, use `Interp2DAmplitude` and for the summation use `InterpolatedModeSum`. **Hint**: You are created a class object that inherits `SchwarzschildEccentricWaveformBase` and initializes it with the proper modules. See the tutorial. It does this exactly.

Documentation:
* [SchwarzschildEccentricWaveformBase](https://bhptoolkit.org/FastEMRIWaveforms/html/user/main.html#few.waveform.SchwarzschildEccentricWaveformBase)
* [Make waveform tutorial](https://bhptoolkit.org/FastEMRIWaveforms/html/tutorial/FastEMRIWaveforms_tutorial.html#Make-a-waveform-model)

In [ ]:
  现在我们将目光转移到一个非常有用的方面：我们可以通过向轨迹中添加项来测试广义相对论效应或外在环境对系统的影响。这里，我们使用一个在之前已经
提供给你的调整好了的轨迹模型。改动的是pdot = pdot * (1 + mod)以及edot = edot * (1 + mod)两个语句。
  它已经放在了下面（ModifiedPnTrajectory）。这是一个在施瓦西偏心极限下基本的牛顿力学近似（PN）轨迹，它可以通过前面的一些教程有效地被掌握。
对于振幅的设置，使用 "Interp2DAmplitude" 模块。
提示：你需要创造一个继承自SchwarzschildEccentricWaveformBase的类对象，并且要用适当的手段进行初始化。去看一下教程，它确实需要你这么做才对。

In [1]:
from pnbeyondGR_example import ModifiedPnTrajectory
from few.amplitude.interp2dcubicspline import Interp2DAmplitude
from few.waveform import SchwarzschildEccentricWaveformBase
from few.summation.interpolatedmodesum import InterpolatedModeSum
from few.trajectory.inspiral import EMRIInspiral

ModuleNotFoundError: No module named 'pnbeyondGR_example'

Create the waveform generator class to start. Look at the FEW tutorial. It has an exact [example](https://bhptoolkit.org/FastEMRIWaveforms/html/tutorial/FastEMRIWaveforms_tutorial.html#Make-a-waveform-model) of how to do this. 

In [ ]:
创建波形发生类来启动程序。看一下FEW这一教程。这里有着关于怎么做的准确例子。

In [ ]:
class FastSchwarzschildEccentricPn(SchEccent):
    def __init__(
        self,
        inspiral_kwargs={},
        amplitude_kwargs={},
        sum_kwargs={},
        Ylm_kwargs={},
        use_gpu=False,
        *args,
        **kwargs
    ):

        SchEccent.__init__(
            self,
            ModifiedPnTrajectory,
            Interp2DAmplitude,
            InterpolatedModeSum,
            inspiral_kwargs=inspiral_kwargs,
            amplitude_kwargs=amplitude_kwargs,
            sum_kwargs=sum_kwargs,
            Ylm_kwargs=Ylm_kwargs,
            use_gpu=use_gpu,
            *args,
            **kwargs
        )

    @property
    def allow_batching(self):
        return False
    @property
    def gpu_capability(self):
        return True

Now use `EMRITDIWaveform` to create a full TDI version of this waveform. **Hint**: use the `emri_waveform_args` keyword argument for the EMRITDIWaveform. Also, use just the A and E channels. Supply the `response_kwargs` keyword argument to specify this. Build the injection waveforms and place them in a `DataResidualArray`. Then, fill an `AnalysisContainer` with the sensitivity matrix, data, and signal generator as we have done before. **Hint**: the modification parameter will be added as the last parameter.

In [ ]:
现在，使用 "EMRITDIWaveform" 来创建一个完整的TDI版本的波形。提示：使用 "emri_waveform_args" 关键字参数来指定 EMRITDIWaveform 的参数。
同时，只使用 A 通道和 E 通道，可以使用 "response_kwargs" 关键字参数来指定它们。构建输入波形并将它们放入一个名为 "DataResidualArray" 的
数据残差数组中，然后创建一个 "AnalysisContainer"，其中包括灵敏度矩阵、数据和信号生成器。与之前相似，可以使用相应的关键字参数来指定这些内容。
提示：修改参数需要充当最后一个添加的参数。

In [ ]:
new_emri_lisa = EMRITDIWaveform(
    emri_waveform_args=(FastSchwarzschildEccentricPn,),
    emri_waveform_kwargs=dict(inspiral_kwargs=dict(func="SchwarzEccFlux")),
    response_kwargs=dict(tdi_chan="AE")
)

Now do some basic calculations (Likelihood, overlap, etc.) to understand how the added effect changes our waveform. **Hint**: you can get the overlap by calculating the inner product with the keyword argument `normalize=True`. 

In [ ]:
  现在做一些基本的计算（似然值、重叠度等）来理解这些我们施加的效应如何改变波形。提示：你可以通过使用normalize=True来计算内积，从而得到重叠
度。

In [ ]:
modification = 0.0

injection = new_emri_lisa(
     M,
    mu,
    a,
    p0,
    e0,
    x0,
    dist,
    qS,
    phiS,
    qK,
    phiK,
    Phi_phi0,
    Phi_theta0,
    Phi_r0,
    modification
)

data = DataResidualArray(injection, dt=dt)
analysis = AnalysisContainer(data, sens_mat, signal_gen=new_emri_lisa)
analysis.calculate_signal_likelihood(
    M,
    mu,
    a,
    p0,
    e0,
    x0,
    dist,
    qS,
    phiS,
    qK,
    phiK,
    Phi_phi0,
    Phi_theta0,
    Phi_r0,
    0.00001,
    source_only=True
)
analysis.calculate_signal_inner_product(
    M,
    mu,
    a,
    p0,
    e0,
    x0,
    dist,
    qS,
    phiS,
    qK,
    phiK,
    Phi_phi0,
    Phi_theta0,
    Phi_r0,
    0.00001,
    normalize=True
)

In [ ]:
收获与总结
  在翻译过程中，刚一看到EMRI就把我整不会了，查阅资料得知，这是极端质量比例旋的意思，绝大多数星系中心都有大质量或超大质量黑洞的存在，当有小质
量的致密星体在运动过程中恰巧接近星系中心的超大质量黑洞时它有可能被黑洞的引力场俘获，结果就是致密星体在新的轨道上围绕着黑洞运动。于是才明白这
一章的研究对象原来是这样一个系统，再往后看，一些处理方法也让我眼前一亮，比如把这样一个系统近似成一个经典力学系统，而把相对论效应和外在的一些影响
视作微扰，如同在量子力学中学过的微扰论一样。总之这些都让我感受到我要做的不仅仅是冷冰冰地编程处理数据，还要深入理解背后的物理图像。